In [2]:
import pandas as pd
import yahooquery as query
from matplotlib import pyplot as plt
import seaborn as sns; sns.set()
import datetime as dt
import numpy as np
%matplotlib

Using matplotlib backend: MacOSX


###### Upsample Dataset Quarterly

# Exploratory Data Analysis

In [350]:
year_start = 2017
year_end = 2020
df = pd.read_csv('BD-PROJECT2M/data/Dataset.csv')
df['asOfDate'] = pd.to_datetime(df['asOfDate'])
df = df.set_index('asOfDate')
tickers = list(df.symbol.drop_duplicates())

In [243]:
def get_prices(ticker_name, start_date, end_date):
  ticker = query.Ticker(ticker_name)
  prices = ticker.history(interval='3mo', start=start_date, end=end_date)[['adjclose']]
  prices = (prices.dropna()).reset_index(level=['symbol', 'date'])
  prices['date'] = pd.to_datetime(prices['date'])
  prices['date'] = prices['date'] - pd.Timedelta(days=1)
  prices = prices.drop(columns=['symbol'])
  prices = prices.rename(columns={'adjclose': ticker_name})
  prices = prices.set_index(['date'])
  return prices[1:]

In [274]:
def merge_dataframes(year_start, year_end):
  start_date = dt.datetime(year_start, 1, 2)
  end_date = dt.datetime(year_end, 12, 31)
  
  base = get_prices(tickers[0], start_date, end_date)
  for ticker in tickers[1:]:
    df = get_prices(ticker, start_date, end_date)
    base = pd.concat([base, df], axis=1)
    #time.sleep(10)
  
  return base

prices = merge_dataframes(year_start, year_end)

In [299]:
stp = (prices[6:].resample('Q', convention='end').asfreq())[:-2]

In [301]:
stp.to_csv('BD-PROJECT2M/data/Dataset_stp.csv')

In [346]:
def get_Y():
    log_returns = np.log(stp).fillna(0)

    data = []

    for col in log_returns.columns:
        lr = pd.DataFrame(log_returns[col])
        lr['symbol'] = col
        lr['log re'] = log_returns[col]
        data.append(lr)

    return ((pd.DataFrame(pd.concat(data)))[['symbol', 'log re']])

y = get_Y().reset_index('date').rename(columns={'date': 'asOfDate'}).set_index('asOfDate')

In [355]:
print(f'X : {df.shape} y : {y.shape}')

X : (1455, 17) y : (1441, 2)


In [365]:
def append_log(x_df,y_df , ticker):
    
    x_df = x_df[x_df['symbol'] == ticker]
    y_df = y_df[y_df['symbol'] == ticker]
    
    df = pd.merge(x_df, y_df, left_index=True, right_index=True)
    df['symbol_x'] = df.rename(columns={'symbol_x': 'symbol'}, inplace=True)
    return df.drop(columns=['symbol_y', 'symbol_x'])

In [378]:
def dataset(x, y):
    
    data = []
    for ticker in tickers:
        data.append(append_log(x,y , ticker))
    return pd.DataFrame(pd.concat(data))

dataset(df, y).to_csv('BD-PROJECT2M/data/Dataset.csv', mode='w+' )